# htFuncLib

In order to use this notebook, first use the [FuncLib](https://funclib.weizmann.ac.il/step/fl_terms/) server to calculate the scores of the different multipoint mutants.   

---


**IMPORTANT** even if the file was changed after FuncLib, only use a file with the original column names and not any other additional ones.


---



Wherever the user's input is required, or a parameter may be changed, it is labeled **EDIT HERE** with an explanation.

## 01 Imports and initial setup

In [ ]:
import numpy as np
import pandas as pd

from tqdm.notebook import tqdm

import matplotlib.patches as mpatches
import matplotlib.pyplot as plt
from matplotlib import rc
import seaborn as sns

from sklearn import preprocessing
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
import joblib

In [ ]:
def display_all(df, n=1000):
    with pd.option_context("display.max_rows", n, "display.max_columns", 1000):
        display(df)

## 02 Loading data from FuncLib server
This is where you use the files you got from FuncLib.

Make sure all runs used the same refined structure and PSSM files (generated in the first FuncLib you ran).



*   ***Minimal number of mutations per design*** was suppose to be set to 1
*   ***Maximal number of mutations per design*** was suppose to be set to 100000 (or the number of positions in your library)
*   ***Difference between clustered variants*** was suppose to be set to 1 (to get all designs calculated by Rosetta)


**EDIT HERE**   
In the following block code, the scores of the different mutants calculated by FuncLib are being loaded. Either upload the files if using Google Colab or edit the file paths if running locally.

In [ ]:
###################
# for google colab
###################
from google.colab import files
import io

uploaded = files.upload() # load the csv files generated by FuncLib
scores = [pd.read_csv(io.BytesIO(v)) for k, v in uploaded.items()]

###################
# for running locally
###################
# paths = [] # add here the paths to the csv files generated by FuncLib
# scores = [pd.read_csv(p) for p in paths]

In [ ]:
def count_mutations(serial_number):
    pairs = [serial_number[i:i+2] for i in range(0, len(serial_number), 2)]
    return sum([1 for i in pairs if i != '01'])

scores = pd.concat(scores, ignore_index=True)

scores.sort_values('total_score', inplace=True)
scores.drop_duplicates(subset=['serial_number'], keep='last', inplace=True)

scores.serial_number = scores.serial_number.apply(lambda x: str(x[1:]))
scores['num_muts'] = scores.serial_number.apply(lambda x: count_mutations(x))

library_positions = set(scores.columns) - set(['serial_number', 'num_muts', 'total_score'])
library_positions = sorted(list(library_positions))

## 03 Prepare data for training

### Sort by total score and find the WT score

In [ ]:
scores.sort_values('total_score', inplace=True)
WT = scores.loc[scores["num_muts"] == 0, :].iloc[0]
assert WT.num_muts == 0

### Set labels

In [ ]:
LABEL = "y"
NLABEL = "negative"
N = len(scores)

In [ ]:
print(WT.total_score)

**EDIT HERE**.   
The parameter ***delta_wt*** can be adjusted to control the selection stringency.

delta_wt > 0 will allow mutants with a total score worse than the WT to be considered "good." We do not recommend using a value higher than 5

delta_wt < 0 will enforce a stringent selection of  "good" multipoint mutants. Use this option if a large part of the enumerated sequence space has a total score better than the WT sequence.


In [ ]:
delta_wt = 0 # edit here (see above)

scores[LABEL] = 0
scores.loc[scores['total_score'] <= WT.total_score + delta_wt, LABEL] = 1

In [ ]:
label1 = sum(scores[LABEL])
percent = 100 * (label1 / N)
print(f'{label1} out of {N} multipoint mutants are labeled as 1 ({percent:.2f})')

We considere the bottom 50% of multipoint mutants to be "bad" designs.

In [ ]:
nlabel = np.zeros(N)
nlabel[int(0.5 * N) :] = 1
scores[NLABEL] = nlabel

In [ ]:
label_rates = pd.DataFrame(columns=["stat", "value"])
label_rates.loc[len(label_rates)] = ["Total number of mutants", N]

label_rates.loc[len(label_rates)] = ['WT score', WT.total_score]
label_rates.loc[len(label_rates)] = ['limit for label 1', WT.total_score + delta_wt]

l1 = len(scores[scores.y == 1])
label_rates.loc[len(label_rates)] = ['Labeled 1',
                                     f'{l1} ({(l1/N)*100:.2f})']

l_neg = len(scores[scores[NLABEL] == 1])
label_rates.loc[len(label_rates)] = ['Labeled negative',
                                     f'{l_neg} ({(l_neg/N)*100:.2f})']

non = len(scores[(scores.y == 0) & (scores.negative == 0)])
label_rates.loc[len(label_rates)] = ['Non classified',
                                     f'{non} ({(non/N)*100:.2f})']
print('Stats on the labeling')
label_rates

In [ ]:
# plot the histogram
plt.figure(facecolor="w")
dist = sns.histplot(
    scores["total_score"],
    color="#99ccff",
    kde=True,
    bins=101,
)

top_negative = min(scores[scores[NLABEL] == 1]["total_score"])
# color less than WT
for p in dist.patches:
    if p.get_x() < WT["total_score"] + delta_wt:
        p.set_facecolor("#66ff66")
    elif p.get_x() >= top_negative:
        p.set_facecolor("#ff6666")


plt.gca().set_yticks([])

patches = []
patches.append(mpatches.Patch(color="#66ff66", label="Labeled good"))
patches.append(mpatches.Patch(color="#99ccff", label="non classified"))
patches.append(mpatches.Patch(color="#ff6666", label="50% bottom"))
plt.legend(handles=patches)
plt.title('Scores distribution')
plt.show()

### Features preparation
Creating the features for the NN to train on. We use one-hot encoding of the data

In [ ]:
encoded_table = pd.get_dummies(scores[library_positions])
encoded_table = encoded_table.loc[:, ~encoded_table.columns.duplicated()]
FEATURES = encoded_table.columns
scores[FEATURES] = encoded_table
scores.head()

### Split to train and test set

**EDIT HERE**


1.   **non labeled** - As the "non labeled" mutants are not sure enough, they are discarded from training.
For using all multipoint mutants in training and testing uncomment the line below
2.   **size of test set** - the size of the test portion of the data can be set using ***test_size***
3.   **random state** - the random state for splitting the data can be set using ***random_state***




In [ ]:
positive = scores[scores[LABEL] == 1]
negative = scores[scores[NLABEL] == 1]
training = pd.concat([positive, negative])
# training = scores # to include the non-labeled mutants in the training data

test_size = 0.2
random_state = 42
train, test = train_test_split(training, test_size=test_size, random_state=random_state)

Leaving only the columns needed for training

In [ ]:
X_train = train[FEATURES]
y_train = pd.DataFrame(train[LABEL], columns=[LABEL])

X_test = test[FEATURES]
y_test = pd.DataFrame(test[LABEL], columns=[LABEL])

print(FEATURES)

### Save Data
Saving all datasets as csv files


In [ ]:
scores.to_csv("all_data.csv")

X_train.to_pickle( "X_train.pkl")
X_test.to_pickle("X_test.pkl")

y_train.to_pickle("y_train.pkl")
y_test.to_pickle("y_test.pkl")

### *Restore* all data and reinitialize whatever variables needed for later

Can be skipped

In [ ]:
scores = pd.read_csv("all_data.csv")

X_train = pd.read_pickle("X_train.pkl")
X_test = pd.read_pickle("X_test.pkl")

y_train = pd.read_pickle("y_train.pkl")
y_test = pd.read_pickle("y_test.pkl")

In [ ]:
FEATURES = X_train.columns
library_positions = sorted(list(set([i.split('_')[0] for i in FEATURES]))) ==

LABEL = "y"
NLABEL = "negative"
N = len(scores)

WT = scores.loc[scores["num_muts"] == 0, :].iloc[1]
assert WT.num_muts == 0

## 04 Training the neural net model

In [ ]:
mlp = MLPClassifier(
    hidden_layer_sizes=(len(library_positions)),
    activation="logistic",
    learning_rate="invscaling",
    random_state=42,
    verbose=True,
    max_iter=20000,
)
mlp.fit(X_train, y_train)

Plotting the loss curve, Should monotonously decline

In [ ]:
plt.figure(facecolor="w")
plt.ion()
ax = plt.gca()
ax.set_title("loss curve")
plt.plot(mlp.loss_curve_)
plt.show()

### Save/ Load the model using joblib
Uncomment the loading model line if restoring a previous run.

In [ ]:
filename = "model.sav"
joblib.dump(mlp, filename)

# mlp = joblib.load(filename)

### Analyzing predictions

In [ ]:
predictions = mlp.predict(scores[FEATURES])

In [ ]:
a = confusion_matrix(scores[LABEL], predictions)
rates = pd.DataFrame(
    [[a[0, 0], a[0, 1], a[1, 0], a[1, 1]]], columns=["True Negative", "False Positive", "False Negative", "True Positive"]
)
rates

In [ ]:
print(classification_report(scores[LABEL], predictions))

## 05 Rank mutaitons by EpiNNet
the following blocks use the EpiNNet model to rank all tested mutations,
refer to the weights_activations_1 column as rank

In [ ]:
pred_diag = pd.DataFrame([{col: 1} for col in FEATURES], columns=FEATURES) # just a diagonal matrix
pred_diag.fillna(0, inplace=True)

weights_activations = mlp.predict_proba(pred_diag)

In [ ]:
results = pd.DataFrame([i.split("_") for i in FEATURES], columns=["position", "AA"])
results["weights_activations_0"] = weights_activations[:, 0]
results["weights_activations_1"] = weights_activations[:, 1]

results.sort_values("weights_activations_1", ascending=False, inplace=True)
results

## 06 Find the best sequence space
These blocks will help you construct the final sequence space.
it iteratively adds additional mutations (from the top ranked mutaitons, as ranked by the EpiNNet model).
for each potential sequence space, the size of the encoded library, and it's estimated good Vs. bad number of deisgns are evaluated.

In [ ]:
def get_seq_space_upto_row(sub_results):
    """
    Get a sequence space of the N top ranked mutations
    :param sub_results: a pandas dataframe of the top ranked mutations
    """
    seq_space = {}
    for p, wt in WT[library_positions].items():
        aas = sub_results.loc[sub_results["position"] == p, "AA"].values
        seq_space[p] = [wt] + [aa for aa in aas if aa != wt]
    return seq_space

def get_sub_score_df(sc_df, seq_space):
    """Get the scores of the mutation spanned by a sequece space"""
    sc_df = sc_df.copy()
    for p, aas in seq_space.items():
        sc_df = sc_df.loc[sc_df[p].isin(aas)]
    return sc_df

In [ ]:
results.sort_values("weights_activations_1", ascending=False, inplace=True)

rows = list()
for i in tqdm(range(len(results))):
    seq_space = get_seq_space_upto_row(results.iloc[:i])
    sub_scores = get_sub_score_df(scores, seq_space)

    row = {p: ''.join(AAs) for p, AAs in seq_space.items()}
    row["i"] = i
    row["good"] = len(sub_scores.loc[sub_scores["y"] == 1])
    row["bad"] = len(sub_scores.loc[sub_scores["negative"] == 1])
    row["total"] = len(sub_scores)
    rows.append(row)

summ_df = pd.DataFrame(rows)

summ_df["lib_size"] = summ_df.apply(lambda r: np.product([len(r[p]) for p in library_positions]), axis=1)
summ_df["rest"] = summ_df.total - summ_df.good - summ_df.bad
summ_df["good_rel"] = summ_df["good"] / summ_df["total"]
summ_df["bad_rel"] = summ_df["bad"] / summ_df["total"]
summ_df["rest_rel"] = summ_df["rest"] / summ_df["total"]
display_all(summ_df)

### Sub-sequence space bar plot
The following bar plot shows the good/bad/unknown rates for all sub sequence spaces, as ranked by EpiNNet

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10, 5))
bottoms = [0] * len(summ_df)

for j, status in enumerate(["good", "bad", "rest"]):
    ax.bar(
        x=summ_df.i,
        height=summ_df[f"{status}_rel"],
        bottom=bottoms,
        width=0.8,
        label=status,
        facecolor=sns.color_palette("pastel", 3)[j],
        edgecolor="k",
    )
    bottoms = [b + s for b, s in zip(bottoms, summ_df[f"{status}_rel"])]
ax.set_xlabel("mutation index")
ax.set_ylabel("frequency in modeled data")
plt.legend()
plt.show()

The above figure shows that the more positions are included, the fraction of bad mutants included in the library is increasing

### Twick the lower and upper limits to find your favorite sequence space
The numbers in the next block will help you examine sequence spaces encoding library sizes in the range you specify.

**EDIT HERE**.    
change the min and max library size and see also the fraction of good and bad designs included

In [ ]:
min_library_size = 1000
max_library_size = 100000
summ_df[summ_df.lib_size.between(min_library_size, max_library_size)][['good_rel', 'bad_rel', 'lib_size']]

**EDIT HERE**.     
Change the variable ***mutation_index*** to see the library generated by the top N residues selected by EpiNNet.

In [ ]:
mutation_index = 20 # change here to different values and see the resulting library

chosen_seq_space = get_seq_space_upto_row(results.iloc[:mutation_index])
sub_scores = get_sub_score_df(scores, chosen_seq_space)

lib_size = np.product(list(map(len, chosen_seq_space.values())))
goods = len(sub_scores.loc[sub_scores["y"] == 1])
bads = len(sub_scores.loc[sub_scores["negative"] == 1])
N = len(sub_scores)

print("position\tAAs")
print('-'*30)
for p, aas in chosen_seq_space.items():
    print(f'{p}\t\t{"".join(aas)}')

print(f"the chosen sequence space encodes a library of size {lib_size:,}")
print(f'the library includes an estimate of {goods/N * 100:.2f}% good designs')
print(f'the library includes an estimate of {bads/N * 100:.2f}% bad designs')

## 07 Test random designs from the library
The library was constructed based on multipoint mutants from the general sequence space, and maybe only seen designs from bubbles and a small percentage of the possible sequence space.

To estimate better the rate of good and bad designs in the chosen library, run a final FuncLib calculation with the selected sequence space.


### Load chosen sequence space mutants

In [ ]:
###################
# for google colab
###################
uploaded_chosen = files.upload() # load the csv files generated by FuncLib
chosen = pd.read_csv(io.BytesIO(list(uploaded_chosen.values())[0]))

###################
# for running locally
###################
# path = '' # add here the path to the csv files generated by FuncLib
# chosen = pd.read_csv(p)

Select a random set of designs from the general sequence space to compare

In [ ]:
N = chosen.shape[0]
general = scores.sample(N)

chosen['seq_space'] = 'chosen'
general['seq_space'] = 'general'
test_10k_df = pd.concat([chosen, general])

Plotting the score distributions. The chosen library will have a distribution shifted to the left (i.e. lower Rosetta scores).

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10, 5), facecolor="w")
sns.kdeplot(
    data=test_10k_df,
    x="total_score",
    hue="seq_space",
    palette="pastel",
    fill=True,
    ax=ax,
    linewidth=2,
)
plt.axvline(WT.total_score, linestyle="--", c="k")

for i, (name, height) in enumerate(zip(["chosen", "general"], [0.8, 0.7])):
    for over_under, left_right in zip(["over", "under"], [0.65, 0.05]):
        if over_under == "under":
            top = len(
                test_10k_df.loc[
                    (test_10k_df["seq_space"] == name)
                    & (test_10k_df["total_score"] < WT.total_score)
                ]
            )
        else:
            top = len(
                test_10k_df.loc[
                    (test_10k_df["seq_space"] == name)
                    & (test_10k_df["total_score"] > WT.total_score)
                ]
            )
        ax.text(
            left_right,
            height,
            f'{100*top/len(test_10k_df.loc[test_10k_df["seq_space"] == name]):.1f}%',
            color=sns.color_palette("pastel")[i],
            size=12,
            transform=ax.transAxes,
        )


plt.show()